### 1. Interpretation and significance

Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

$$ expenditure = 873 + 0.0012annual\_income + 0.00002annual\_income^2 - 223.57have\_kids $$

*expenditure* is the annual spending on recreation in US dollars, *annual_income* is the annual income in US dollars, and *have_kids* is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.

### Answer 1: 

Assuming the coefficients are statistically significant (which we don't know because none of them have their associated statistics reported), we can say that the bias term is $873, the expenditure increases in quadratically with annual income, and families with kids spend 223.57 dollars fewer on recreation.

### 2. Weather model

In this exercise, you'll work with the historical temperature data from the previous checkpoint. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* First, load the dataset from the **weatherinszeged** table from Thinkful's database.
* Build a linear regression model where your target variable is the difference between the *apparenttemperature* and the *temperature*. As explanatory variables, use *humidity* and *windspeed*. Now, estimate your model using OLS. Are the estimated coefficients statistically significant? Are the signs of the estimated coefficients in line with your previous expectations? Interpret the estimated coefficients. What are the relations between the target and the explanatory variables? 
* Next, include the interaction of *humidity* and *windspeed* to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for *humidity* and *windspeed* change? Interpret the estimated coefficients.

### Answer 2:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sqlalchemy import create_engine

# Display preferences.
%matplotlib inline
pd.options.display.float_format = "{:.3f}".format

%load_ext nb_black

import warnings

warnings.filterwarnings(action="ignore")

<IPython.core.display.Javascript object>

In [2]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "weatherinszeged"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
szeged_df = pd.read_sql_query("select * from weatherinszeged", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


szeged_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472,7.389,0.890,14.120,251.000,15.826,0.000,1015.130,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.356,7.228,0.860,14.265,259.000,15.826,0.000,1015.630,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.378,9.378,0.890,3.928,204.000,14.957,0.000,1015.940,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.289,5.944,0.830,14.104,269.000,15.826,0.000,1016.410,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.756,6.978,0.830,11.045,259.000,15.826,0.000,1016.510,Partly cloudy throughout the day.


<IPython.core.display.Javascript object>

In [3]:
szeged_df.isna().mean()

date                  0.000
summary               0.000
preciptype            0.000
temperature           0.000
apparenttemperature   0.000
humidity              0.000
windspeed             0.000
windbearing           0.000
visibility            0.000
loudcover             0.000
pressure              0.000
dailysummary          0.000
dtype: float64

<IPython.core.display.Javascript object>

In [4]:
szeged_df.describe()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure
count,96453.000,96453.000,96453.000,96453.000,96453.000,96453.000,96453.000,96453.000
mean,11.933,10.855,0.735,10.811,187.509,10.347,0.000,1003.236
std,9.552,10.697,0.195,6.914,107.383,4.192,0.000,116.970
min,-21.822,-27.717,0.000,0.000,0.000,0.000,0.000,0.000
25%,4.689,2.311,0.600,5.828,116.000,8.340,0.000,1011.900
50%,12.000,12.000,0.780,9.966,180.000,10.046,0.000,1016.450
75%,18.839,18.839,0.890,14.136,290.000,14.812,0.000,1021.090
max,39.906,39.344,1.000,63.853,359.000,16.100,0.000,1046.380


<IPython.core.display.Javascript object>

In [5]:
szeged_df["target_temp"] = szeged_df["apparenttemperature"] - szeged_df["temperature"]

<IPython.core.display.Javascript object>

In [6]:
szeged_df.describe()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,target_temp
count,96453.000,96453.000,96453.000,96453.000,96453.000,96453.000,96453.000,96453.000,96453.000
mean,11.933,10.855,0.735,10.811,187.509,10.347,0.000,1003.236,-1.078
std,9.552,10.697,0.195,6.914,107.383,4.192,0.000,116.970,1.679
min,-21.822,-27.717,0.000,0.000,0.000,0.000,0.000,0.000,-10.183
25%,4.689,2.311,0.600,5.828,116.000,8.340,0.000,1011.900,-2.217
50%,12.000,12.000,0.780,9.966,180.000,10.046,0.000,1016.450,0.000
75%,18.839,18.839,0.890,14.136,290.000,14.812,0.000,1021.090,0.000
max,39.906,39.344,1.000,63.853,359.000,16.100,0.000,1046.380,4.811


<IPython.core.display.Javascript object>

In [7]:
y = szeged_df["target_temp"]
X = szeged_df[["humidity", "windspeed"]]

<IPython.core.display.Javascript object>

In [8]:
X = sm.add_constant(X)

results = sm.OLS(y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            target_temp   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Wed, 08 Apr 2020   Prob (F-statistic):               0.00
Time:                        23:53:34   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

The coefficients are statistically significant at p < 0.001.  The difference in apparent versus regular temperature (wind chill?), i.e. target_temp, seems to decrease with increase in humidity or windspeed.  For a one unit increase in humidity, the target_temp decreases by 3.03 degrees.  For a one unit increase in windspeed, the target_temp decreases by 0.012 degrees.

In [9]:
szeged_df["wind_hum_interaction"] = szeged_df["windspeed"] * szeged_df["humidity"]

<IPython.core.display.Javascript object>

In [10]:
y = szeged_df["target_temp"]
X2 = szeged_df[["humidity", "windspeed", "wind_hum_interaction"]]

<IPython.core.display.Javascript object>

In [11]:
X2 = sm.add_constant(X2)

results = sm.OLS(y, X2).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            target_temp   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Wed, 08 Apr 2020   Prob (F-statistic):               0.00
Time:                        23:53:35   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0839      0.033      2.511      0.012       0.018       0.149
humidity                 0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                0.0905      0.002     36.797      0.000       0.086       0.095
wind_hum_interaction    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

All of the coefficients are still statistically significant at the p < 0.01 level.  

The signs of the humidity and windspeed coefficients are now positive.  The interaction variable has a negative sign and a larger magnitude than either of the individual coefficients.

For a given windspeed, a 1 point increase in humidity has a .17-0.3X windspeed point change in the target variable.  For a given humidity, a one point increase in windspeed results in a 0.09-0.3X humidity point change in the target.

###  3. House prices model

In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?
* Now, exclude the insignificant features from your model. Did anything change?
* Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?
* Do the results sound reasonable to you? If not, try to explain the potential reasons.

### Answer 3:

In [12]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

import statsmodels.stats.api as sms
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

%load_ext nb_black

import warnings

warnings.filterwarnings("ignore")

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [13]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
houseprices_df = pd.read_sql_query("select * from houseprices", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


houseprices_df.head(5)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

In [14]:
houseprices_df["log_price"] = np.log(houseprices_df["saleprice"])

<IPython.core.display.Javascript object>

In [15]:
houseprices_df = houseprices_df[
    [
        "log_price",
        "overallqual",
        "grlivarea",
        "garagearea",
        "fullbath",
        "yearbuilt",
        "exterqual",
        "garagefinish",
        "centralair",
    ]
]

<IPython.core.display.Javascript object>

In [16]:
hp_df = houseprices_df.dropna()

<IPython.core.display.Javascript object>

In [17]:
drop_ids = hp_df.sort_values("grlivarea", ascending=False).iloc[:4].index
print(f"Dropping house ids {list(drop_ids)}")
hp_df = hp_df.drop(index=drop_ids)

Dropping house ids [1298, 523, 1182, 691]


<IPython.core.display.Javascript object>

In [18]:
hp_df["centralair"] = (hp_df["centralair"] == "Y").astype(int)

<IPython.core.display.Javascript object>

In [19]:
quality_map = {"Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}
hp_df[["exterqual"]] = hp_df[["exterqual"]].replace(quality_map)

<IPython.core.display.Javascript object>

In [20]:
garage_df = pd.get_dummies(hp_df["garagefinish"], drop_first=True)

<IPython.core.display.Javascript object>

In [21]:
hp_df = pd.concat([hp_df, garage_df], axis=1)
hp_df.drop(columns=["garagefinish"], inplace=True)

<IPython.core.display.Javascript object>

In [22]:
hp_df

,log_price,overallqual,grlivarea,garagearea,fullbath,yearbuilt,exterqual,centralair,RFn,Unf
0,12.248,7,1710,548,2,2003,3,1,1,0
1,12.109,6,1262,460,2,1976,2,1,1,0
2,12.317,7,1786,608,2,2001,3,1,1,0
3,11.849,7,1717,642,1,1915,2,1,0,1
4,12.429,8,2198,836,2,2000,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
1455,12.073,6,1647,460,2,1999,2,1,1,0
1456,12.255,6,2073,500,2,1978,2,1,0,1
1457,12.493,7,2340,252,2,1941,4,1,1,0
1458,11.864,5,1078,240,1,1950,2,1,0,1


<IPython.core.display.Javascript object>

In [25]:
X = hp_df.drop(columns=["log_price", "RFn"])
y = hp_df["log_price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=32
)

<IPython.core.display.Javascript object>

In [26]:
X_train_const = sm.add_constant(X_train)
lm = sm.OLS(y_train, X_train_const).fit()

lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     692.4
Date:                Wed, 08 Apr 2020   Prob (F-statistic):               0.00
Time:                        23:57:09   Log-Likelihood:                 501.86
No. Observations:                1100   AIC:                            -985.7
Df Residuals:                    1091   BIC:                            -940.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.0321      0.492     14.297      0.000       6.067       7.997
overallqual     0.0901      0.006     15.229      0.000       0.079       0.102
grlivarea       0.0003   1.47e-05     21.794      0.000       0.000       0.000
garagearea      0.0003   3.25e-05      8.230      0.000       0.000       0.000
fullbath       -0.0422      0.013     -3.275      0.001      -0.068      -0.017
yearbuilt       0.0018      0.000      7.051      0.000       0.001       0.002
exterqual       0.0716      0.012      5.770      0.000       0.047       0.096
centralair      0.1788      0.023      7.746      0.000       0.134       0.224
Unf            -0.0586      0.012     -4.737      0.000      -0.083      -0.034
==============================================================================
Omnibus:                      112.703   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              311.139
Skew:                          -0.537   Prob(JB):                     2.74e-68
Kurtosis:                       5.374   Cond. No.                     2.71e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.71e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>

In the previous assignment, I already discarded the features from my model that were statistically insignificant.  These features were:
        "log_price",
        "garagecars",
        "totrmsabvgrd",
        "CBlock",
        "PConc",
        "Stone",
        "Wood",
        "RFn". When I did exclude these features and re-ran the model, the R-squared values didn't really change.  Neither did the coefficients of the features that are still present in the model.
        
The estimated model is:

$$ ln(price) = 7.0321 + 0.0901overallqual + 0.0003(grlivearea + garagearea) - 0.0422fullbath + 0.0018yearbuilt + 0.0716exterqual + 0.1788centralair - 0.0586Unf \qquad(eq.1)$$

The feature with by far the most prominent positive affect on price is the presence of central air-conditioning.   These results sound reasonable to me.  